In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from laspy.file import File

import sys
sys.path.insert(0,'..')
from point_density_functions import *

In [6]:
# Load nyc dataframe
usgs_file_dir = '../../Data/USGS/'
usgs_pt_file = list(pd.read_csv(usgs_file_dir+"usgs_pt_files.txt",header=None)[0])
usgs_df = pd.read_hdf(usgs_file_dir+usgs_pt_file[0])

#Sort by gps_time
usgs_df.sort_values(by=['gps_time'],inplace=True)
usgs_df = usgs_df.drop(0)


In [7]:
usgs_df = usgs_df.reset_index()


In [8]:
def label_returns(las_df):
    '''
    Parses the flag_byte into number of returns and return number, adds these fields to las_df.
    Input - las_df - dataframe from .laz or .lz file
    Output - first_return_df - only the first return points from las_df.
           - las_df - input dataframe with num_returns and return_num fields added 
    '''
    
    las_df['num_returns'] = np.floor(las_df['return_number_byte']/8).astype(int)
    las_df['return_num'] = las_df['return_number_byte']%8
    first_return_df = las_df[las_df['return_num']==1]
    first_return_df = first_return_df.reset_index(drop=True)
    return first_return_df, las_df
# Note: this anlaysis doesn't really need this
_,usgs_df = label_returns(usgs_df)

In [9]:
gap_times = np.array(usgs_df.iloc[1:]['gps_time']) - np.array(usgs_df.iloc[:-1]['gps_time'])

In [10]:
# Gaps between flights
gap_times[gap_times>30]

array([1.01443393e+03, 3.81407438e+02, 4.61488490e+05])

In [12]:
# Add flight_id for nyc flights based on time gaps
usgs_df['flight_id'] = [0]*usgs_df.shape[0]
flight_id = 0
old_i = 0
for i,gap in enumerate(gap_times):
    if gap > 30:
        usgs_df.loc[old_i:i+1,'flight_id'] = flight_id
        flight_id+=1
        old_i = i
        print("New group!")
usgs_df.loc[old_i:,'flight_id'] = flight_id

New group!
New group!
New group!


In [22]:
# Create hdf file
usgs_df.to_hdf(usgs_file_dir+'las_points_flight_id_18TWK820985.lz',key='df',complevel=1,complib='lzo')